In [1]:
# Data Handling
import numpy as np
import pandas as pd
import seaborn as sns

# Visualization
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.patches as mpatches
import seaborn as sns
import plotly.express as px

# Statistical Analysis
from statsmodels.tsa.stattools import adfuller
from scipy.stats import ttest_ind
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_squared_log_error


# Feature Processing
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler

# Modelling
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.model_selection import RandomizedSearchCV
import pickle

# Other Packages
import warnings

warnings.filterwarnings("ignore")



In [2]:
train_df = pd.read_csv("data/store-sales-time-series-forecasting/train.csv")
test_df = pd.read_csv("data/store-sales-time-series-forecasting/test.csv")
stores_df = pd.read_csv("data/store-sales-time-series-forecasting/stores.csv")
oil_df = pd.read_csv("data/store-sales-time-series-forecasting/oil.csv")
holidays_events_df = pd.read_csv("data/store-sales-time-series-forecasting/holidays_events.csv")
transactions_df = pd.read_csv("data/store-sales-time-series-forecasting/transactions.csv")

In [3]:
# Converting the 'date' column in the datasets to datetime format
# Train dataset
train_df['date'] = pd.to_datetime(train_df['date'])

# Test dataset
test_df['date'] = pd.to_datetime(test_df['date'])

# Holiday Events dataset
holidays_events_df['date'] = pd.to_datetime(holidays_events_df['date'])

# Oil dataset
oil_df['date'] = pd.to_datetime(oil_df['date'])

# Transactions dataset
transactions_df['date'] = pd.to_datetime(transactions_df['date'])

In [4]:
print(f"Train Dataset: {train_df.shape}")
print(f"Test Datasets: {test_df.shape}")

Train Dataset: (3000888, 6)
Test Datasets: (28512, 5)


In [5]:
# Confirm the data type of the 'date' column after transformation
print('Date Column Data Type After Transformation:') 
print('==='*14)
print("Train dataset:", train_df['date'].dtype)
print("Test dataset:", test_df['date'].dtype)
print("Holiday Events dataset:", holidays_events_df['date'].dtype)
print("Oil dataset:", oil_df['date'].dtype)
print("Transactions dataset:", transactions_df['date'].dtype)

Date Column Data Type After Transformation:
Train dataset: datetime64[ns]
Test dataset: datetime64[ns]
Holiday Events dataset: datetime64[ns]
Oil dataset: datetime64[ns]
Transactions dataset: datetime64[ns]


In [6]:
# Fill missing values in the 'dcoilwtico' column using backfill strategy
oil_df['dcoilwtico'] = oil_df['dcoilwtico'].fillna(method='backfill')

Created the test.csv file

In [7]:
# Check the completeness of the test dataset
min_date = test_df['date'].min()
max_date = test_df['date'].max()
expected_dates = pd.date_range(start=min_date, end=max_date)

missing_dates = expected_dates[~expected_dates.isin(test_df['date'])]

if len(missing_dates) == 0:
    print("The test dataset is complete. It includes all the required dates.")
else:
    print("The test dataset is incomplete. The following dates are missing:")
    print(missing_dates)

The test dataset is complete. It includes all the required dates.


In [8]:
# Merging the common columns ('store_nbr' and 'date') in the datasets using the inner merge() function
# Merge train_df with stores_df based on 'store_nbr' column
merged_df_test = test_df.merge(stores_df, on='store_nbr', how='left')

# Merge merged_df1 with transactions_df based on 'date' and 'store_nbr' columns
# merged_df2 = merged_df_test.merge(transactions_df, on=['date', 'store_nbr'], how='left')

# # Merge merged_df2 with holidays_events_df based on 'date' column
merged_df_test = merged_df_test.merge(holidays_events_df, on='date', how='left')

# # Merge merged_df3 with oil_df based on 'date' column
# merged_df4_test = merged_df3_test.merge(oil_df, on='date', how='left')

# # View the first five rows of the merged dataset
merged_df_test.head()

,id,date,store_nbr,family,onpromotion,city,state,type_x,cluster,type_y,locale,locale_name,description,transferred
0,3000888,2017-08-16,1,AUTOMOTIVE,0,Quito,Pichincha,D,13,NaN,NaN,NaN,NaN,NaN
1,3000889,2017-08-16,1,BABY CARE,0,Quito,Pichincha,D,13,NaN,NaN,NaN,NaN,NaN
2,3000890,2017-08-16,1,BEAUTY,2,Quito,Pichincha,D,13,NaN,NaN,NaN,NaN,NaN
3,3000891,2017-08-16,1,BEVERAGES,20,Quito,Pichincha,D,13,NaN,NaN,NaN,NaN,NaN
4,3000892,2017-08-16,1,BOOKS,0,Quito,Pichincha,D,13,NaN,NaN,NaN,NaN,NaN


In [9]:
# Renaming the columns with the appropriate names
merged_df_test = merged_df_test.rename(columns={"type_x": "store_type", "type_y": "holiday_type"})
merged_df_test.head()

,id,date,store_nbr,family,onpromotion,city,state,store_type,cluster,holiday_type,locale,locale_name,description,transferred
0,3000888,2017-08-16,1,AUTOMOTIVE,0,Quito,Pichincha,D,13,NaN,NaN,NaN,NaN,NaN
1,3000889,2017-08-16,1,BABY CARE,0,Quito,Pichincha,D,13,NaN,NaN,NaN,NaN,NaN
2,3000890,2017-08-16,1,BEAUTY,2,Quito,Pichincha,D,13,NaN,NaN,NaN,NaN,NaN
3,3000891,2017-08-16,1,BEVERAGES,20,Quito,Pichincha,D,13,NaN,NaN,NaN,NaN,NaN
4,3000892,2017-08-16,1,BOOKS,0,Quito,Pichincha,D,13,NaN,NaN,NaN,NaN,NaN


In [10]:
merged_df_test['date'] = pd.to_datetime(merged_df_test['date'])
merged_df_test['year'] = merged_df_test['date'].dt.year

In [11]:
merged_df_test['month'] = merged_df_test['date'].dt.month
merged_df_test['day'] = merged_df_test['date'].dt.day

In [12]:
merged_df_test['day_of_week'] = merged_df_test['date'].dt.dayofweek

In [13]:
merged_df_test.head()

,id,date,store_nbr,family,onpromotion,city,state,store_type,cluster,holiday_type,locale,locale_name,description,transferred,year,month,day,day_of_week
0,3000888,2017-08-16,1,AUTOMOTIVE,0,Quito,Pichincha,D,13,NaN,NaN,NaN,NaN,NaN,2017,8,16,2
1,3000889,2017-08-16,1,BABY CARE,0,Quito,Pichincha,D,13,NaN,NaN,NaN,NaN,NaN,2017,8,16,2
2,3000890,2017-08-16,1,BEAUTY,2,Quito,Pichincha,D,13,NaN,NaN,NaN,NaN,NaN,2017,8,16,2
3,3000891,2017-08-16,1,BEVERAGES,20,Quito,Pichincha,D,13,NaN,NaN,NaN,NaN,NaN,2017,8,16,2
4,3000892,2017-08-16,1,BOOKS,0,Quito,Pichincha,D,13,NaN,NaN,NaN,NaN,NaN,2017,8,16,2


In [14]:
merged_df_test.shape

(28512, 18)

In [15]:
columns_to_drop = ['date','id', 'locale', 'locale_name', 'description', 'store_type', 'transferred', 'state']
merged_df_test = merged_df_test.drop(columns=columns_to_drop)

merged_df_test.head()

,store_nbr,family,onpromotion,city,cluster,holiday_type,year,month,day,day_of_week
0,1,AUTOMOTIVE,0,Quito,13,NaN,2017,8,16,2
1,1,BABY CARE,0,Quito,13,NaN,2017,8,16,2
2,1,BEAUTY,2,Quito,13,NaN,2017,8,16,2
3,1,BEVERAGES,20,Quito,13,NaN,2017,8,16,2
4,1,BOOKS,0,Quito,13,NaN,2017,8,16,2


In [16]:
merged_df_test.shape

(28512, 10)

In [17]:
unique_families = merged_df_test['family'].unique()
unique_families

array(['AUTOMOTIVE', 'BABY CARE', 'BEAUTY', 'BEVERAGES', 'BOOKS',
       'BREAD/BAKERY', 'CELEBRATION', 'CLEANING', 'DAIRY', 'DELI', 'EGGS',
       'FROZEN FOODS', 'GROCERY I', 'GROCERY II', 'HARDWARE',
       'HOME AND KITCHEN I', 'HOME AND KITCHEN II', 'HOME APPLIANCES',
       'HOME CARE', 'LADIESWEAR', 'LAWN AND GARDEN', 'LINGERIE',
       'LIQUOR,WINE,BEER', 'MAGAZINES', 'MEATS', 'PERSONAL CARE',
       'PET SUPPLIES', 'PLAYERS AND ELECTRONICS', 'POULTRY',
       'PREPARED FOODS', 'PRODUCE', 'SCHOOL AND OFFICE SUPPLIES',
       'SEAFOOD'], dtype=object)

In [18]:
# Define the category lists for each product category
food_families = ['BEVERAGES', 'BREAD/BAKERY', 'FROZEN FOODS', 'MEATS', 'PREPARED FOODS', 'DELI','PRODUCE', 'DAIRY','POULTRY','EGGS','SEAFOOD']
home_families = ['HOME AND KITCHEN I', 'HOME AND KITCHEN II', 'HOME APPLIANCES']
clothing_families = ['LINGERIE', 'LADYSWARE']
grocery_families = ['GROCERY I', 'GROCERY II']
stationery_families = ['BOOKS', 'MAGAZINES','SCHOOL AND OFFICE SUPPLIES']
cleaning_families = ['HOME CARE', 'BABY CARE','PERSONAL CARE']
hardware_families = ['PLAYERS AND ELECTRONICS','HARDWARE']

# Categorize the 'family' column based on the product categories
merged_df_test['family'] = np.where(merged_df_test['family'].isin(food_families), 'FOODS', merged_df_test['family'])
merged_df_test['family'] = np.where(merged_df_test['family'].isin(home_families), 'HOME', merged_df_test['family'])
merged_df_test['family'] = np.where(merged_df_test['family'].isin(clothing_families), 'CLOTHING', merged_df_test['family'])
merged_df_test['family'] = np.where(merged_df_test['family'].isin(grocery_families), 'GROCERY', merged_df_test['family'])
merged_df_test['family'] = np.where(merged_df_test['family'].isin(stationery_families), 'STATIONERY', merged_df_test['family'])
merged_df_test['family'] = np.where(merged_df_test['family'].isin(cleaning_families), 'CLEANING', merged_df_test['family'])
merged_df_test['family'] = np.where(merged_df_test['family'].isin(hardware_families), 'HARDWARE', merged_df_test['family'])

# Print the updated DataFrame
merged_df_test.head()

,store_nbr,family,onpromotion,city,cluster,holiday_type,year,month,day,day_of_week
0,1,AUTOMOTIVE,0,Quito,13,NaN,2017,8,16,2
1,1,CLEANING,0,Quito,13,NaN,2017,8,16,2
2,1,BEAUTY,2,Quito,13,NaN,2017,8,16,2
3,1,FOODS,20,Quito,13,NaN,2017,8,16,2
4,1,STATIONERY,0,Quito,13,NaN,2017,8,16,2


In [19]:
# Define the categorical columns to encode
categorical_columns = ["family", "city"]

# Perform one-hot encoding
encoder = OneHotEncoder()
one_hot_encoded_test_data = encoder.fit_transform(merged_df_test[categorical_columns])

# Create column names for the one-hot encoded data
column_names = encoder.get_feature_names_out(categorical_columns)

# Convert the one-hot encoded data to a DataFrame
merged_df_test_encoded = pd.DataFrame(one_hot_encoded_test_data.toarray(), columns=column_names)

# Concatenate the original dataframe with the one-hot encoded data
merged_df_test_encoded = pd.concat([merged_df_test, merged_df_test_encoded], axis=1)

# Drop the original categorical columns
merged_df_test_encoded.drop(categorical_columns, axis=1, inplace=True)

# Print the head of the encoded DataFrame
merged_df_test_encoded.head()

,store_nbr,onpromotion,cluster,holiday_type,year,month,day,day_of_week,family_AUTOMOTIVE,family_BEAUTY,...,city_Loja,city_Machala,city_Manta,city_Playas,city_Puyo,city_Quevedo,city_Quito,city_Riobamba,city_Salinas,city_Santo Domingo
0,1,0,13,NaN,2017,8,16,2,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,1,0,13,NaN,2017,8,16,2,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,1,2,13,NaN,2017,8,16,2,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,1,20,13,NaN,2017,8,16,2,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,1,0,13,NaN,2017,8,16,2,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [20]:
holidays = ['holiday_type_Additional', 'holiday_type_Bridge', 'holiday_type_Event', 'holiday_type_Holiday', 'holiday_type_Transfer', 'holiday_type_Work Day']

In [21]:
for holiday in holidays:
    
    merged_df_test_encoded[holiday] = 0

In [24]:
merged_df_test_encoded = merged_df_test_encoded.drop(columns="holiday_type")

In [25]:
merged_df_test_encoded.shape

(28512, 49)

In [26]:
merged_df_test_encoded.columns

Index(['store_nbr', 'onpromotion', 'cluster', 'year', 'month', 'day',
       'day_of_week', 'family_AUTOMOTIVE', 'family_BEAUTY',
       'family_CELEBRATION', 'family_CLEANING', 'family_CLOTHING',
       'family_FOODS', 'family_GROCERY', 'family_HARDWARE', 'family_HOME',
       'family_LADIESWEAR', 'family_LAWN AND GARDEN',
       'family_LIQUOR,WINE,BEER', 'family_PET SUPPLIES', 'family_STATIONERY',
       'city_Ambato', 'city_Babahoyo', 'city_Cayambe', 'city_Cuenca',
       'city_Daule', 'city_El Carmen', 'city_Esmeraldas', 'city_Guaranda',
       'city_Guayaquil', 'city_Ibarra', 'city_Latacunga', 'city_Libertad',
       'city_Loja', 'city_Machala', 'city_Manta', 'city_Playas', 'city_Puyo',
       'city_Quevedo', 'city_Quito', 'city_Riobamba', 'city_Salinas',
       'city_Santo Domingo', 'holiday_type_Additional', 'holiday_type_Bridge',
       'holiday_type_Event', 'holiday_type_Holiday', 'holiday_type_Transfer',
       'holiday_type_Work Day'],
      dtype='object')

In [27]:
merged_df_test_encoded.to_csv("test_modified.csv", index=False)

In [27]:
pd.set_option('display.max_columns', None)

In [28]:
merged_df_test_encoded.head()

,store_nbr,onpromotion,cluster,year,month,day,day_of_week,family_AUTOMOTIVE,family_BEAUTY,family_CELEBRATION,family_CLEANING,family_CLOTHING,family_FOODS,family_GROCERY,family_HARDWARE,family_HOME,family_LADIESWEAR,family_LAWN AND GARDEN,"family_LIQUOR,WINE,BEER",family_PET SUPPLIES,family_STATIONERY,city_Ambato,city_Babahoyo,city_Cayambe,city_Cuenca,city_Daule,city_El Carmen,city_Esmeraldas,city_Guaranda,city_Guayaquil,city_Ibarra,city_Latacunga,city_Libertad,city_Loja,city_Machala,city_Manta,city_Playas,city_Puyo,city_Quevedo,city_Quito,city_Riobamba,city_Salinas,city_Santo Domingo
0,1,0,13,2017,8,16,2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,1,0,13,2017,8,16,2,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,1,2,13,2017,8,16,2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,1,20,13,2017,8,16,2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,1,0,13,2017,8,16,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [39]:
hol = pd.read_csv("data/store-sales-time-series-forecasting/holidays_events.csv")

In [42]:
hol.type.unique()

array(['Holiday', 'Transfer', 'Additional', 'Bridge', 'Work Day', 'Event'],
      dtype=object)